In [ ]:
# this is a test
print("Hello World")
print("test test test")

In [ ]:
print("test")
print("Gabby test")
print("Emily test")
print("gabby test 2!")
# hello this is emily and i'm so cool

In [ ]:
print("gabriella test")
# this is a test
# use arrays to make clue and answer the tie the arrays together
# def create-questions(clue, answer): with the clue being the questions
# quesions = [] for i in clue: questions.append ((clue,answer))
# hello i am testing this code right now

In [ ]:
# starts the dictionaries 
#exoplanets first
import numpy as np

def create_dictionary(clue, answer, is_daily_double, point_value, category):
    for i in range(30):
        tile = {
            "clue": clue[i],
            "answer": answer[i],
            "is_daily_double": is_daily_double[i],
            "point_value": point_value[i],
            "category": category[i]
        }



In [49]:
# Makes an array 

your_mom = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
desired_shape = (3, 4)
array_2d = np.array(your_mom).reshape(desired_shape)
print(array_2d)



[[ 1  2  3  4]
 [ 5  6  7  8]
 [ 9 10 11 12]]


In [ ]:
# Makes an array with the list
list = []
for i in range(30):
    list.append(i)

desired_shape = (5, 6)
array_2d = np.array(list).reshape(desired_shape)
print(array_2d)


[[ 0  1  2  3  4  5]
 [ 6  7  8  9 10 11]
 [12 13 14 15 16 17]
 [18 19 20 21 22 23]
 [24 25 26 27 28 29]]


In [24]:
import numpy as np
#1. define categories and Questions
categories = ["exoplanets", "black holes", "galaxies", "stars", "solar system"]
# 5 rows (values: 200-1000)
board_values = np.array([100, 200, 300, 400, 500])

# 2. Randomly pick a Daily Double location using choice
dd_selection = np.random.choice(30)

# Convert that one number into specific row (0-4) and col (0-5)
dd_row = dd_selection // 6
dd_col = dd_selection % 6

print(f"Daily Double Location: Row {dd_row}, Col {dd_col} (Category: {categories[dd_col]})")

# 3. Initialize Board with placeholders (5 rows x 6 columns)
board = [["Q" for _ in range(6)] for _ in range(5)]
board[dd_row][dd_col] = "DD" # Mark Daily Double

# 4. Display Board
print("\nJeopardy Board:")
for row in board:
    print(row)

Daily Double Location: Row 1, Col 4 (Category: solar system)

Jeopardy Board:
['Q', 'Q', 'Q', 'Q', 'Q', 'Q']
['Q', 'Q', 'Q', 'Q', 'DD', 'Q']
['Q', 'Q', 'Q', 'Q', 'Q', 'Q']
['Q', 'Q', 'Q', 'Q', 'Q', 'Q']
['Q', 'Q', 'Q', 'Q', 'Q', 'Q']
